<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/novel_data/%E5%B0%8F%E8%AF%B4%E6%8A%BD%E5%8F%96%E4%B8%BAchunk%EF%BC%88%E5%89%8D%E5%A4%84%E7%90%86%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [ ] 解压缩一个小说part包到content
- [ ] 把单个小说到chunks的函数封装
- [ ] 汇总一个jsonl到/content
- [ ] 尝试在GPU下压缩看看能有多大

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
part_id = 0
# input_file = f"/content/drive/MyDrive/Wuxia/input/parts_{part_id}.zip"

input_file = f"/content/drive/MyDrive/Wuxia/input/classic_50.zip"


In [ ]:
import os
import zipfile
import shutil

output_dir = "/content/"

# 解压input_file到output_dir
with zipfile.ZipFile(input_file,"r") as zip_ref:
    zip_ref.extractall(output_dir)

In [ ]:
import os
import chardet

output_folder = "/content/output2"

file_names = []

# 遍历所有txt文件
for root, dirs, files in os.walk(output_folder):
    for file in files:
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)

            with open(file_path, 'rb') as f:
                content = f.read(1000)
            result = chardet.detect(content)
            # 检测编码
            # result = chardet.detect(open(file_path, 'rb').read())

            # 判断是否GB2312或UTF-8
            # if result['encoding'] in ['GB2312', 'UTF-8']:
            if result['confidence'] < 0.9:
                print(f"Warning: {file_path} encoding confidence {result['confidence']:.0%} lower than 90%")
                continue

            file_names.append({
                "file_name": file_path,
                "encoding": result['encoding'],
                "confidence": result['confidence']
            })

In [ ]:
print(file_names)

[{'file_name': '/content/output2/《秦时明月》（精校1-8部）作者：温世仁.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《间客》（精校全本）作者：猫腻.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《娘要嫁人》（精校全本）作者：严歌苓.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《狼图腾》（校对全本）作者：姜戎.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《红楼遗梦》（校对全本）作者：冬雪晚晴.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《明朝那些事儿》（精校全本）作者：当年明月.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《冰与火之歌03冰雨的风暴》（精校全本）作者：乔治·R·R·马丁.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《步步惊心》（精校全本）作者：桐华.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《藏地密码》（精校全本）作者：何马.txt', 'encoding': 'GB2312', 'confidence': 0.99}, {'file_name': '/content/output2/《绾青丝》（校对全本）作者：波波.txt', 'encoding': 'GB2312', 'confiden

In [ ]:
import re

def get_file_name(fname):
    last_split = fname.split('/')[-1]

    pattern = r'《(.+)》'
    m = re.match(pattern, last_split)
    if m:
        # print("match")
        return m.group(1) + '.txt'
    else:
        return last_split

fname = "/content/output/output2/《凡人修仙传》(精校全本)作者:忘语.txt"

print(get_file_name(fname)[:-4])

凡人修仙传


切分小说

In [ ]:
def predict_chapater(line):
    line = line.strip()
    line = line.strip('-=*')
    if len(line) > 100:
        return -1
    if len(line) == 0:
        return -1
    is_short = len(line) < 10

    head_5_char = line[:min(5,len(line))]

    small_line = line.lower()
    line = line[:20]
    has_di = line.find('第', 0 ) >= 0
    di_in_head = head_5_char.find('第', 0) >= 0

    has_zhang = False
    zhang_in_head = False

    zhang_word = ['章', '集', '卷','回','期','节']



    for w in zhang_word:
        if head_5_char.find(w, 0) >= 0:
            zhang_in_head = True
            break

    for w in zhang_word:
        if line.find(w, 0) >= 0:
            has_zhang = True
            break

    has_digital = False
    digital_word = ['0','1','2','3','4','5','6','7','8','9',\
                    '一','二','三','四','五','六','七','八','九']

    only_digital = False
    digital_count = 0
    for ch in line:
        if ch in digital_word:
            digital_count += 1
    if digital_count == len(line):
        only_digital = True

    digital_in_head = False

    for w in digital_word:
        if head_5_char.find(w, 0) >= 0:
            digital_in_head = True
            break
    for w in digital_word:
        if line.find(w, 0) >= 0:
            has_digital = True
            break
    if has_di and has_zhang and has_digital and (di_in_head or digital_in_head or zhang_in_head):
        return 3
    if di_in_head and digital_in_head and is_short:
        return 2
    if zhang_in_head and digital_in_head and is_short:
        return 2
    # if only_digital and is_short:
    #     return 2
    return -1

#@title  定义divide函数，用来切分超长文本
def divide_str(s, sep=['\n', '.', '。']):
    mid_len = len(s) // 2  # 中心点位置
    best_sep_pos = len(s) + 1  # 最接近中心点的分隔符位置
    best_sep = None  # 最接近中心点的分隔符
    for curr_sep in sep:
        sep_pos = s.rfind(curr_sep, mid_len // 2, len(s)-mid_len // 2)  # 从中心点往左找分隔符
        if sep_pos > 0 and abs(sep_pos - mid_len) < abs(best_sep_pos -
                                                        mid_len):
            best_sep_pos = sep_pos
            best_sep = curr_sep
    if not best_sep:  # 没有找到分隔符
        return s, ''
    return s[:best_sep_pos + 1], s[best_sep_pos + 1:]


def strong_divide(s):
    left, right = divide_str(s)

    if right != '':
        return left, right

    whole_sep = ['\n', '.', '，', '、', ';', ',', '；',\
                 '：', '！', '？', '(', ')', '”', '“', \
                 '’', '‘', '[', ']', '{', '}', '<', '>', \
                 '/', '''\''', '|', '-', '=', '+', '*', '%', \
               '$', '''#''', '@', '&', '^', '_', '`', '~',\
                 '·', '…']
    left, right = divide_str(s, sep=whole_sep)

    if right != '':
        return left, right

    mid_len = len(s) // 2
    return s[:mid_len], s[mid_len:]


In [ ]:
import codecs

for file_name_data in file_names:
    file_name = file_name_data['file_name']
    if file_name.find("大染坊",0)<0:
        # 先找一本进行调试
        continue

    break


In [ ]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
import tiktoken
enc= tiktoken.encoding_for_model("gpt-3.5-turbo")

In [ ]:
file_name = "/content/output2/《大染坊》（精校全本）作者：陈杰.txt"

book_name = get_file_name(file_name)
if len(book_name) > 4:
    book_name = book_name[:-4]

fencoding = file_name_data['encoding']
f = codecs.open(file_name, 'r', encoding=fencoding, errors = 'ignore')
raw_text = f.read()
f.close()



In [ ]:
def book2chunks(raw_text, book_name):
    lines = raw_text.splitlines()

    is_heads = [predict_chapater(line) > 0 for line in lines]

    # token_per_line = []

    # from tqdm import tqdm

    token_per_line = [len(enc.encode(line)) for line in lines]

    chunks = []

    current_n = 0
    current_chunk = ""
    current_chunk_is_head = True
    last_chunk_is_head = False

    TOKEN_PER_TRUNK = 1000

    id_count = 0

    for line, is_head, n_token in zip(lines, is_heads, token_per_line):
        # 如果is_head且之前不是head
        # 如果is_head且之前的current_n > TOKEN_PER_TRUNK // 2 需要结算
        # 如果n_token + current_n > TOKEN_PER_TRUNK 需要结算
        count_flag = False

        if is_head and not last_chunk_is_head:
            count_flag = True
        elif is_head and last_chunk_is_head and current_n > TOKEN_PER_TRUNK // 2:
            count_flag = True
        elif n_token + current_n > TOKEN_PER_TRUNK:
            count_flag = True

        if count_flag and current_chunk.strip() != "":
            chunk_data = {
                "text": current_chunk,
                "is_head": current_chunk_is_head,
                "id": book_name + "_" + str(id_count)
            }
            id_count += 1
            chunks.append(chunk_data)
            last_chunk_is_head = current_chunk_is_head
            current_chunk_is_head = is_head
            current_chunk = ""
            current_n = 0

        current_chunk += line + "\n"
        current_n += n_token

    if current_chunk.strip() != "":
        chunk_data = {
            "text": current_chunk,
            "is_head": current_chunk_is_head,
            "id": book_name + "_" + str(id_count)
        }
        chunks.append(chunk_data)

    for chunk in chunks:
        text = chunk['text']
        text = text.replace("\u3000", "\n")
        for i in range(3):
            text = text.replace("\n\n", "\n")
        # text = text.replace("\n\n", "\n")
        chunk['text'] = text.strip()
        del chunk['is_head']
    return chunks

chunks = book2chunks(raw_text, book_name)

In [ ]:
save_datas = []

from tqdm import tqdm

for file_name_data in tqdm(file_names):
    file_name = file_name_data['file_name']

    book_name = get_file_name(file_name)
    if len(book_name) > 4:
        book_name = book_name[:-4]

    fencoding = file_name_data['encoding']
    f = codecs.open(file_name, 'r', encoding=fencoding, errors = 'ignore')
    raw_text = f.read()
    f.close()

    chunks = book2chunks(raw_text, book_name)

    save_datas.extend(chunks)

100%|██████████| 51/51 [01:21<00:00,  1.59s/it]


In [ ]:
print(len(save_datas))

118997


In [ ]:
import json

save_name = "/content/classic_50.jsonl"

with open(save_name, 'w') as f:
    for data in save_datas:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")

请为我实现一段python程序

将 /content/classic_50.jsonl

压缩到classic_50.zip

并且print压缩后的文件的大小

In [ ]:
# 首先，将指定的 JSONL 文件压缩为 ZIP 文件
# 然后，计算并打印压缩后的文件大小

import zipfile
import os

# 指定要压缩的文件路径和目标 ZIP 文件路径
source_file_path = '/content/classic_50.jsonl'
target_zip_path = '/content/classic_50.zip'

# 使用 zipfile 模块创建 ZIP 文件并添加指定的文件
with zipfile.ZipFile(target_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(source_file_path, os.path.basename(source_file_path))

# 计算并打印压缩后的文件大小
compressed_file_size = os.path.getsize(target_zip_path)
print(compressed_file_size)


96417330


为我实现一段python程序，将/content/classic_50.zip

拷贝到/content/drive/MyDrive/CardBuild/exp0210/dialogue_extract_input

In [ ]:
import shutil

# 定义源 ZIP 文件路径和目标目录路径
source_zip_path = '/content/classic_50.zip'
target_directory_path = '/content/drive/MyDrive/CardBuild/exp0210/dialogue_extract_input/classic_50.zip'

# 尝试拷贝文件
# 注意：这里的路径和操作基于一个假设的环境，实际上在当前环境中不可能直接访问 `/content` 目录
# 因此，我们将模拟这个过程的逻辑，但不会实际执行

# 模拟的过程
shutil.copy(source_zip_path, target_directory_path)

'/content/drive/MyDrive/CardBuild/exp0210/dialogue_extract_input/classic_50.zip'

In [ ]:
import shutil

# 定义源 ZIP 文件路径和目标目录路径
source_zip_path = '/content/classic_50.jsonl'
target_directory_path = '/content/drive/MyDrive/CardBuild/exp0210/dialogue_extract_input/classic_50.jsonl'

# 尝试拷贝文件
# 注意：这里的路径和操作基于一个假设的环境，实际上在当前环境中不可能直接访问 `/content` 目录
# 因此，我们将模拟这个过程的逻辑，但不会实际执行

# 模拟的过程
shutil.copy(source_zip_path, target_directory_path)

'/content/drive/MyDrive/CardBuild/exp0210/dialogue_extract_input/classic_50.jsonl'

In [ ]:
print(len(save_datas))
print(save_datas[0].keys())

118997
dict_keys(['text', 'id'])


In [ ]:
import random
for _ in range(3):
    random_id = random.randint(0, len(save_datas) - 1)
    print(save_datas[random_id]['id'])
    print(save_datas[random_id]['text'])
    print('---')

超级兵王_1830
张士天不说监狱还好，一说监狱就让郭璞心头憋火，自己被冤枉关了四年，他妈的，这笔账正好找不到人算呢，你这孙子居然敢说管小爷一辈子！
“呵呵！这位大爷！你想关我一辈子？不过你不行！尤其是像你们这群蛆虫，渣滓！或者都是多余的王八蛋，我要是你们的爹，当时就直接把你们射在墙上了！现在回炉也晚了！不过好在还有机会，我可以给你们一个重新做人的机会！”
张士天和文化还有洪志东加上其他的十几个二世祖同时呆了！
静！
周围死一般的安静，只有海风在轻轻地吹动着树梢！
所有的人都张大着嘴巴看着眼前这个笑嘻嘻说出一大堆叫人忍不住勃然变色的话的家伙！
那些二世祖们一时之间也没能反应过来，郭璞呵呵一笑，继续说道：“这位大爷，难道我说错了？”
看到郭璞的又在说话，二世祖们才反应过来。纷纷嚷嚷着指着郭璞就是一阵的开骂！
“哪里来的狗东西？居然敢这样羞辱我们？”
“就是，把他做了，他妈的，又不是没杀过人！”
“狗东西，居然敢羞辱张公子，真是在找死啊！”
“上，打死这个狗东西！”
郭璞突然哈哈大笑起来，看着被一群二世祖围在中间的张士天，眼里突然闪过一道残忍的杀意，但是没有人看到他的眼神！
“好吧好吧！我是狗东西！不过这位张公子，你可要看好了！这五个美女，都是我的女人，你确定你还要继续惹我吗？”
“呸！你的女人？真是大言不惭！”
“就是，我看这五个女人就是夜总会的小姐！他妈的，估计是上海哪个夜总会的！嘿嘿，居然跑到我们的地盘上来了！兄弟们，有口福了！那个归我，就是胸最大的那个女人，哈哈，正有气质！”
“对，那我要那个冷冰冰的，哈哈，好久没玩这么冷的女人了！看看有什么不一样饿滋味！”
米兰气得柳眉倒竖，但是随即她就是眉眼一转，装着害怕的样子，走到郭璞的身边，颤声说道：“老板，救救我们！我怕！！”
“老板，我也怕！”
“老板！我怕怕！”
---
凡人修仙传_6972
第一千五百三十九章 大难逃脱
“是不是简道友弄错了，那新出现的玄天之宝并未在我们飞灵族区域。如此的话，无法用血祭之术召来此宝也是正常之事。”一名淡淡的女子声音也从祭坛上响起。
说话之人却是一名身披白色斗篷的中年美妇，背后一对羽翅五色闪亮。刚才的言语中，明显带有一丝幸灾乐祸之意。
“步夫人！刚才召唤法阵是我和简兄一起主持的，通过血祭之力的确都感应到了那件玄天斩灵剑的存在。但是在传回时似乎出了什么差错，竟然在途中忽然消失了。难道此